# Extracting Data From Nofrills API

In [1]:
import json
import requests
import time
import pandas as pd
import os
from datetime import datetime

In [2]:
BASE_URL = 'https://api.pcexpress.ca/pcx-bff/api/v2/listingPage/28195'

In [3]:
HEADER = {
    'origin' : 'https://www.nofrills.ca',
    'referer' : 'https://www.nofrills.ca/',
    'user-agent' : 'Mozilla/5.0',
    'Content-Type': 'application/json',
    "accept": "application/json",
    'x-apikey' : 'C1xujSegT5j3ap3yexJjqhOfELwGKYvz',
    'x-application-type' : 'web',
    'x-channel' : 'web',
    'accept-language' : 'en',
    'x-loblaw-tenant-id' : 'ONLINE_GROCERIES'
}


In [4]:
BASE_PAYLOAD = {
    "cart": {
        "cartId": "fe034ffc-1d6f-4ebc-a9a4-46bf57d686cb"
    },
    "userData": {
        "domainUserId": "217d777e-b431-4c6e-ac02-ff921a11e85f",
        "sessionId": "3d4c913a-b394-4834-aec2-7416d95a3d62"
    },
    "fulfillmentInfo": {
        "offerType": "OG",
        "storeId": "7496",
        "pickupType": "STORE",
        "date": "06012026",
        "timeSlot": None
    },
    "banner": "nofrills",
    "listingInfo": {
        "filters": {
            "navid": ["flyout-L2-fruits-vegetables"],
            "icid": ["gr_fruits-vegetables-shop-categories_tile_3_hp"],
            "cartId": ["0"]
        },
        "sort": {},
        "pagination": {
            "from": 1
        },
        "includeFiltersInResponse": True
    },
    "options": [
        {"name": "bff.exp.EnableNextGenClpPlp", "value": "variant"},
        {"name": "bff.exp.flyerIABTest", "value": "variant"},
        {"name": "bff.exp.EnableNextGenSRP", "value": "variant"},
        {"name": "bff.exp.TEMP_TEST_FIX", "value": "variant"},
        {"name": "bff.exp.EnableNextGenFlyerItems", "value": "variant"},
        {"name": "bff.exp.EnableNextGenClp", "value": "variant"},
        {"name": "bff.exp.coldstart", "value": "variant"},
        {"name": "bff.exp.srp_viewability", "value": "variant"}
    ],
    "device": {
        "screenSize": 1470
    }
}


In [5]:
rows = []
page_size = 48  # Number of items per page
start_index = 1  # Offset starts at 0
extracted_at = datetime.utcnow().isoformat()
source = "No_frills_Api"

while True:
    BASE_PAYLOAD["listingInfo"]["pagination"]["from"] = start_index
    response = requests.post(url=BASE_URL, headers=HEADER, json=BASE_PAYLOAD, timeout=30)
    response.raise_for_status()

    raw = response.json()
    
    components = raw.get("layout", {}).get("sections", {}).get("productListingSection", {}).get("components",[])
    for component in components:
        productGrid = component.get("data",{}).get("productGrid",{})
        break

    productTiles = productGrid.get('productTiles',[])
    if not productTiles:
        break
    
    for productTile in productTiles:
        rows.append({
            "payload" : json.dumps(productTile),
            "extracted_at" : extracted_at,
            'source' : "No_frills_Api"
        })

    start_index += 1




    
    

In [10]:
df= pd.DataFrame(rows)
df.head(5)

,payload,extracted_at,source
0,"{""promotions"": [], ""productId"": ""20003839001_K...",2026-01-07T02:46:23.350153,No_frills_Api
1,"{""promotions"": [], ""productId"": ""20088387001_K...",2026-01-07T02:46:23.350153,No_frills_Api
2,"{""promotions"": [], ""productId"": ""20150181001_K...",2026-01-07T02:46:23.350153,No_frills_Api
3,"{""promotions"": [], ""productId"": ""20043212001_K...",2026-01-07T02:46:23.350153,No_frills_Api
4,"{""promotions"": [], ""productId"": ""20096584001_K...",2026-01-07T02:46:23.350153,No_frills_Api


In [32]:
test = df.iloc[1,0]
test

'{"promotions": [], "productId": "20088387001_KG", "productImage": [{"imageUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_1200.png", "thumbnailUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_120.png", "smallUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_250.png", "mediumUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_400.png", "largeUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_800.png", "extraLargeUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_1200.png", "smallRetinaUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_250.png", "mediumRetinaUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_400.png", "largeRetinaUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_800.png"}], "brand": null, "title": "Jalapeno Peppers",

In [ ]:
'{"promotions": [],'
' "productId": "20088387001_KG", "productImage": [{"imageUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_1200.png", "thumbnailUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_120.png", "smallUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_250.png", "mediumUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_400.png", "largeUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_800.png", "extraLargeUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_1200.png", "smallRetinaUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_250.png", "mediumRetinaUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_400.png", "largeRetinaUrl": "https://digital.loblaws.ca/PCX/20088387001_KG/en/1/20088387001_en_front_800.png"}], '
'"brand": null, '
'"title": "Jalapeno Peppers", '
'"description": "Jalapeno peppers are slightly rounded chilies with smooth green skin that turns red when ripe. Jalapenos are popular because of their flavour and because they are easy to seed. Add jalapeno peppers to sauces and salsas. Scoville scale rating of 2,500 \\u2013 8,000.", '
'"packageSizing": "$11.00/1kg $4.99/1lb", '
'"pricing": {"price": "0.55", "wasPrice": null, "displayPrice": "about $0.55", "memberOnlyPrice": null, "mopDisplayPrice": null, "ehfTotal": null, "environmentalHandlingFee": null, "taxInfo": ""},'
' "pricingUnits": {"type": "SOLD_BY_EACH_PRICED_BY_WEIGHT", "unit": "ea", "interval": 1, "minOrderQuantity": 1, "maxOrderQuantity": 24, "weighted": false}, "deal": null, "mopDeal": null, "pcoDeal": null, "inventoryIndicator": null, "offerType": "OG", "offerId": "OG", "action": null, "isVariant": false, "link": "/jalapeno-peppers/p/20088387001_KG", "productVendor": null, "productVariant": null, "textBadge": "", "variantTheme": null, "articleNumber": "20088387001", "uom": "KG", "sponsoredCreative": null, "isSponsored": false, "isComplementarySponsored": false, "complementarySponsoredBadgeText": null, "productBadge": null}'

# Exporting Data to Azure

In [17]:
import os
import io
from azure.storage.blob import BlobServiceClient
import pyarrow
from dotenv import load_dotenv

In [23]:
load_dotenv()
Account =os.environ.get('ACCOUNT_NAME')
SAS_TOKEN = os.environ.get('SAS_TOKEN')
CONTAINER_NAME = os.environ.get('CONTAINER_NAME')
BLOB_NAME = os.environ.get('BLOB_NAME_NO_FRILLS')

In [25]:
account_url = f'https://{Account}.blob.core.windows.net'
blobclient = BlobServiceClient(
    account_url=account_url,
    credential= SAS_TOKEN
)

In [26]:
parquet_buffer = io.BytesIO()
df.to_parquet(parquet_buffer, engine= 'pyarrow', index=False)
parquet_buffer.seek(0)

0

In [27]:
blobupload = blobclient.get_blob_client(
    container=CONTAINER_NAME,
    blob=BLOB_NAME
)
blobupload.upload_blob(parquet_buffer, overwrite=True)


{'etag': '"0x8DE4D97799280FC"',
 'last_modified': datetime.datetime(2026, 1, 7, 2, 50, 10, tzinfo=datetime.timezone.utc),
 'content_md5': bytearray(b'\xfe\x06\xcf\xfbuQ\xa5\x8aDU\xd3\xa2\xfd\xcf\xdbC'),
 'client_request_id': '94f93220-eb73-11f0-bdeb-b685ec340e58',
 'request_id': 'a84c869a-301e-0064-4980-7f7afb000000',
 'version': '2025-11-05',
 'version_id': None,
 'date': datetime.datetime(2026, 1, 7, 2, 50, 9, tzinfo=datetime.timezone.utc),
 'request_server_encrypted': True,
 'encryption_key_sha256': None,
 'encryption_scope': None,
 'structured_body': None}